<a href="https://colab.research.google.com/github/yongsa-nut/SF251_67_2/blob/main/SF251_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cohere API and LLM Basic

In [ ]:
!pip install cohere

In [ ]:
from google.colab import userdata
import cohere

# Get your free API key: https://dashboard.cohere.com/api-keys
co = cohere.ClientV2(api_key=userdata.get('cohere'))

Basic API Usage

In [ ]:
# Add the user message
message = "Hello!"

# Add the messages
messages = [
    {"role": "user", "content": message}
]

# Generate the response
response = co.chat(model="command-a-03-2025",
                   messages=messages)

print(response.message.content[0].text)

Other example: https://openrouter.ai/

Conversation Loop:
- User -> Assistant -> User -> Assistant -> ...

In [ ]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write a short introduction message to my teammates."

# Create a custom system message
system_message = """## Task and Context
Generate concise responses, with maximum one-sentence."""

# Add the messages
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": message},
]

# Generate the response
response = co.chat(model="command-a-03-2025", messages=messages)

print(response.message.content[0].text)

Then add the assistant's response to our messages and we can add the user's message next.

In [ ]:
# Append the previous response
messages.append(
    {"role": "assistant", "content": response.message.content[0].text}
)
# Add the user message
message = "Make it more upbeat and conversational."
# Append the user message
messages.append({"role": "user", "content": message})
# Generate the response with the current chat history as the context
response = co.chat(model="command-a-03-2025", messages=messages)
print(response.message.content[0].text)

Then we can keep going.
Below is the full loop.

In [ ]:
system_message = "You are a helpful assistant." # << Change this to see the effect

messages = [{"role": "system", "content": system_message}]

# Coding this together!



In [ ]:
print(messages)

# RAG

- Note: Cohere actually has its own RAG api ([documentation](https://docs.cohere.com/docs/rag-with-cohere))

<br>

---

## Demo 1: Keyword Matching

In [ ]:
def get_response(query):
  response = co.chat(model="command-a-03-2025",
                     messages=[{'role':'user','content':query}])
  return response.message.content[0].text

get_response('Hello test test')

In [ ]:
# The data
lorebook = { 'CN101' : 'Introduction to Computer Programming หลักการพื้นฐานคอมพิวเตอร์ องค์ประกอบคอมพิวเตอร์การท างานร่วมกันฮาร์ดแวร์และซอฟต์แวร์ การเขียนโปรแกรมภาษาคอมพิวเตอร์ การฝึกฝนการเขียนโปรแกรมคอมพิวเตอร์',
             'MA111' : 'Fundamentals of Calculus อุปนัยเชิงคณิตศาสตร์ ระบบจ านวนและฟังก์ชันเบื้องต้น แคลคูลัสอนุพันธ์และปริพันธ์ของฟังก์ชัน ตัวแปรเดียว ลิมิต ความต่อเนื่อง อนุพันธ์และการประยุกต์อนุพันธ์ ปฏิยานุพันธ์ เทคนิคการหาปริพันธ์และ การประยุกต์ปริพันธ์ ปริพันธ์ไม่ตรงแบบ อนุกรม ทฤษฎีบทเทย์เลอร์ส าหรับฟังก์ชันพื้นฐาน การหาปริพันธ์เชิงตัวเลข',
             'TSE100' : 'Ethics for Engineers จรรยาบรรณวิศวกรรม ผลกระทบของเทคโนโลยีต่อสังคม ปัญหาและประเด็นทางด้านจริยธรรม และคุณธรรมแนวทางแก้ไขตลอดจนการป้องกัน เพื่อไม่ให้เกิดกรณีดังกล่าวกับลักษณะงานทางวิศวกรรม ด้านต่างๆ การเข้าร่วมโครงการอบรมจริยธรรม เพื่อพัฒนาคุณธรรมและจริยธรรม วัดผลเป็นระดับ S หรือ U (เข้าร่วมกิจกรรมกับที่ทางคณะวิศวกรรมศาสตร์จัดขึ้น)'
}

In [ ]:
def keyword_generate(query, docs):
  # Retrive information and create context
  context = '<context>'
  for k in docs:
    if k in query:
      context += f'{k} = {docs[k]}\n'
  context += '</context>'

  prompt = f'''<question>{query}</question>
  Please use context in <context> tags to answer the question.
  <context>{context}</context>'''
  response = get_response(prompt)
  # printing out
  print('Query:', query)
  print('Retrieved documents:', context)
  print('Response:', response)

query = 'CN101 เรียนอะไร?'

keyword_generate(query, lorebook)

<br>

---

## Demo 2: BM25

In [ ]:
# Install necessary libraries
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
# Sample document collection
# Curriculum: https://drive.google.com/file/d/1dofsvV5XptwgnXDZIyo5Y4vOmkl8v5qr/view
documents = [
    "SF210 Programming Methodology: An introduction to the engineering of computer software emphasizing modern software engineering principles: object-oriented design, decomposition,encapsulation, abstraction, and testing.",
    "SF211 Object-Oriented Programming: Introduction to object-oriented programming. Class, Object, Encapsulation, Inheritance, Polymorphism, and Abstraction.",
    "CN200 Discrete Mathematics: Logic. Proof techniques. Basic set theory. Relations and functions. Mathematical induction. Countability and counting arguments. Permutations and combinations. Inclusionexclusion principle. Elementary finite probability. Topics in graph theory: isomorphism, planarity, circuits, trees, and directed graphs.",
    "SF230 Linear Algebra and Numerical Analysis: Theorems of matrices, vector spaces, linear independence, dimensions, rank of matrices, applications of matrices for solving systems of linear equations, inverse of matrices, determinant, Cramer’ s Rule, linear transformations, inner product spaces, orthogonal complement and least square, eigenvalues, eigenvectors and its application. Numerical solutions of one variable equations, polynomial interpolation, numerical methods of differentiation and integration, solving engineering problems by using package",
    "SF250 Probability Theory and Statistics: Introduction to probability theory. Topics covered include random variables, conditional probability, expectation, independence, Bayes' rule, important distributions, joint distributions, central limit theorem, laws of large numbers, statistical inference; point and confidence interval estimation, hypothesis tests, analysis of variance, linear regression.",
    "SF212 Web Application Development: Introduction to the basic principles of web application programming. Web server systems. Basic HTML and Cascading StyleSheets. Server-side web application development. Database access and manipulation through the web. Session management. Web application security.",
    "SF220 Introduction to Software Engineering: Scientific foundation for software engineering, introduction to software development process and life cycles. Methods, techniques, and tools used for software engineering process. Students work in small teams on substantial, realistic projects, covering most phases of the software production life cycle.",
    "SF231 Data Structures and Algorithms: Introduction to data structures and algorithms, algorithm analysis, arrays and linked lists, stacks, queues, priority queues, heaps, binary trees, binary search trees, AVL trees, other variations in trees, hashing, sorting, graph algorithms, algorithm design techniques, online judges and algorithm competitions.",
    "SF251 Introduction to Data Science: Learning the foundational principles of data science and data mining through an in-depth examination of the data science lifecycle. This comprehensive course delves into each important steps, beginning with a comprehension of problem formulation tailored to the specific needs and objectives of businesses. Learners will engage in data mining methodologies, including processes of data extraction, transformation, and integration. Moreover, an analysis of data quality and refinement techniques will be undertaken to ensure the integrity and reliability of datasets. Furthermore, learners will explore the complexities of feature engineering, unraveling intricate relationships and patterns within the data domain. Leveraging state-of-the-art machine learning algorithms, learners will delve into predictive analytics, forecasting future trends and behaviors with precision and accuracy. Additionally, the course will emphasize the art of data presentations, enabling participants to effectively communicate analytical insights to diverse stakeholders.",
    "CN351 Web Application Security: Current state of security in web applications. Key security mechanisms for web applications. Client and server side controls. Common vulnerabilities of web-based applications and how to protect against the attacks."
    ]

# Simple tokenization using string splitting
tokenized_docs = [doc.lower().split() for doc in documents]

# Create BM25 object
bm25 = BM25Okapi(tokenized_docs)

In [ ]:
def retrieve(query, top_k=2):
  # Words only
  tokenized_query = query.lower().split()
  # Pass the list of words into bm25 to get scores
  doc_scores = bm25.get_scores(tokenized_query)
  # Then retrieve the score
  top_docs = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:top_k]

  return [documents[i] for i, _ in top_docs]

In [ ]:
def bm25_response(query):
  retrieved_docs = retrieve(query)
  context = "\n".join(retrieved_docs)

  prompt = f'''<question>{query}</question>
  Please use context in <context> tags to answr the question.
  <context>{context}</context>'''
  response = get_response(prompt)
  # printing out
  print('Query:', query)
  print('Retrieved documents:', retrieved_docs)
  print('\nResponse:', response)

query = "What do you learn in Data Science ?"
bm25_response(query)

<br>

---

## Demo 3: RAG without actual database

In [ ]:
!pip install sentence_transformers datasets

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Same document collection
documents = [
    "SF210 Programming Methodology: An introduction to the engineering of computer software emphasizing modern software engineering principles: object-oriented design, decomposition,encapsulation, abstraction, and testing.",
    "SF211 Object-Oriented Programming: Introduction to object-oriented programming. Class, Object, Encapsulation, Inheritance, Polymorphism, and Abstraction.",
    "CN200 Discrete Mathematics: Logic. Proof techniques. Basic set theory. Relations and functions. Mathematical induction. Countability and counting arguments. Permutations and combinations. Inclusionexclusion principle. Elementary finite probability. Topics in graph theory: isomorphism, planarity, circuits, trees, and directed graphs.",
    "SF230 Linear Algebra and Numerical Analysis: Theorems of matrices, vector spaces, linear independence, dimensions, rank of matrices, applications of matrices for solving systems of linear equations, inverse of matrices, determinant, Cramer’ s Rule, linear transformations, inner product spaces, orthogonal complement and least square, eigenvalues, eigenvectors and its application. Numerical solutions of one variable equations, polynomial interpolation, numerical methods of differentiation and integration, solving engineering problems by using package",
    "SF250 Probability Theory and Statistics: Introduction to probability theory. Topics covered include random variables, conditional probability, expectation, independence, Bayes' rule, important distributions, joint distributions, central limit theorem, laws of large numbers, statistical inference; point and confidence interval estimation, hypothesis tests, analysis of variance, linear regression.",
    "SF212 Web Application Development: Introduction to the basic principles of web application programming. Web server systems. Basic HTML and Cascading StyleSheets. Server-side web application development. Database access and manipulation through the web. Session management. Web application security.",
    "SF220 Introduction to Software Engineering: Scientific foundation for software engineering, introduction to software development process and life cycles. Methods, techniques, and tools used for software engineering process. Students work in small teams on substantial, realistic projects, covering most phases of the software production life cycle.",
    "SF231 Data Structures and Algorithms: Introduction to data structures and algorithms, algorithm analysis, arrays and linked lists, stacks, queues, priority queues, heaps, binary trees, binary search trees, AVL trees, other variations in trees, hashing, sorting, graph algorithms, algorithm design techniques, online judges and algorithm competitions.",
    "SF251 Introduction to Data Science: Learning the foundational principles of data science and data mining through an in-depth examination of the data science lifecycle. This comprehensive course delves into each important steps, beginning with a comprehension of problem formulation tailored to the specific needs and objectives of businesses. Learners will engage in data mining methodologies, including processes of data extraction, transformation, and integration. Moreover, an analysis of data quality and refinement techniques will be undertaken to ensure the integrity and reliability of datasets. Furthermore, learners will explore the complexities of feature engineering, unraveling intricate relationships and patterns within the data domain. Leveraging state-of-the-art machine learning algorithms, learners will delve into predictive analytics, forecasting future trends and behaviors with precision and accuracy. Additionally, the course will emphasize the art of data presentations, enabling participants to effectively communicate analytical insights to diverse stakeholders.",
    "CN351 Web Application Security: Current state of security in web applications. Key security mechanisms for web applications. Client and server side controls. Common vulnerabilities of web-based applications and how to protect against the attacks."
    ]

# Initialize sentence transformer model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed documents
doc_embeddings = embed_model.encode(documents)

In [ ]:
doc_embeddings

In [ ]:
len(doc_embeddings[0])

In [ ]:
def retrieve_RAG_docs(query, top_k=1):
    # Embed the query
    query_embedding = embed_model.encode([query])

    # Calculate cosine similarity
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Get top-k relevant documents
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [documents[i] for i in top_indices]

In [ ]:
def RAG_response(query, top_k=3):
  retrieved_docs = retrieve_RAG_docs(query, top_k)
  context = "\n".join(retrieved_docs)

  prompt = f'''<question>{query}</question>
  Please use context in <context> tags to answr the question.
  <context>{context}</context>'''
  response = get_response(prompt)
  # printing out
  print('Query:', query)
  print('Retrieved documents:', retrieved_docs)
  print('\nResponse:', response)

query = "What do you learn in data science?"
RAG_response(query)

<br>

---

## Demo 4: RAG with Pinecone

In [ ]:
!pip3 install pinecone

In [ ]:
from pinecone import Pinecone, ServerlessSpec

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [ ]:
query = 'What jumps over the dog?'
xq = model.encode(query)
xq.shape

In [ ]:
# Same document collection
documents = [
    "SF210 Programming Methodology: An introduction to the engineering of computer software emphasizing modern software engineering principles: object-oriented design, decomposition,encapsulation, abstraction, and testing.",
    "SF211 Object-Oriented Programming: Introduction to object-oriented programming. Class, Object, Encapsulation, Inheritance, Polymorphism, and Abstraction.",
    "CN200 Discrete Mathematics: Logic. Proof techniques. Basic set theory. Relations and functions. Mathematical induction. Countability and counting arguments. Permutations and combinations. Inclusionexclusion principle. Elementary finite probability. Topics in graph theory: isomorphism, planarity, circuits, trees, and directed graphs.",
    "SF230 Linear Algebra and Numerical Analysis: Theorems of matrices, vector spaces, linear independence, dimensions, rank of matrices, applications of matrices for solving systems of linear equations, inverse of matrices, determinant, Cramer’ s Rule, linear transformations, inner product spaces, orthogonal complement and least square, eigenvalues, eigenvectors and its application. Numerical solutions of one variable equations, polynomial interpolation, numerical methods of differentiation and integration, solving engineering problems by using package",
    "SF250 Probability Theory and Statistics: Introduction to probability theory. Topics covered include random variables, conditional probability, expectation, independence, Bayes' rule, important distributions, joint distributions, central limit theorem, laws of large numbers, statistical inference; point and confidence interval estimation, hypothesis tests, analysis of variance, linear regression.",
    "SF212 Web Application Development: Introduction to the basic principles of web application programming. Web server systems. Basic HTML and Cascading StyleSheets. Server-side web application development. Database access and manipulation through the web. Session management. Web application security.",
    "SF220 Introduction to Software Engineering: Scientific foundation for software engineering, introduction to software development process and life cycles. Methods, techniques, and tools used for software engineering process. Students work in small teams on substantial, realistic projects, covering most phases of the software production life cycle.",
    "SF231 Data Structures and Algorithms: Introduction to data structures and algorithms, algorithm analysis, arrays and linked lists, stacks, queues, priority queues, heaps, binary trees, binary search trees, AVL trees, other variations in trees, hashing, sorting, graph algorithms, algorithm design techniques, online judges and algorithm competitions.",
    "SF251 Introduction to Data Science: Learning the foundational principles of data science and data mining through an in-depth examination of the data science lifecycle. This comprehensive course delves into each important steps, beginning with a comprehension of problem formulation tailored to the specific needs and objectives of businesses. Learners will engage in data mining methodologies, including processes of data extraction, transformation, and integration. Moreover, an analysis of data quality and refinement techniques will be undertaken to ensure the integrity and reliability of datasets. Furthermore, learners will explore the complexities of feature engineering, unraveling intricate relationships and patterns within the data domain. Leveraging state-of-the-art machine learning algorithms, learners will delve into predictive analytics, forecasting future trends and behaviors with precision and accuracy. Additionally, the course will emphasize the art of data presentations, enabling participants to effectively communicate analytical insights to diverse stakeholders.",
    "CN351 Web Application Security: Current state of security in web applications. Key security mechanisms for web applications. Client and server side controls. Common vulnerabilities of web-based applications and how to protect against the attacks."
    ]

doc_embeddings = embed_model.encode(documents)

### Setup Pinecone

In [ ]:
# Get secret key
from google.colab import userdata

In [ ]:
pinecone = Pinecone(api_key=userdata.get('pinecone_key'))
INDEX_NAME = 'test1-5-2-2025'

# Cleaning up the index
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)

# Creating a serverless index
pinecone.create_index(
    name = INDEX_NAME,
    dimension = model.get_sentence_embedding_dimension(),
    metric = 'cosine',
    spec = ServerlessSpec(cloud='aws', region='us-east-1')) #

index = pinecone.Index(INDEX_NAME)
print(index)

### Upsert to Pinecone

- Format: A list of dict
  - `{'id':xx, 'value':embedding, 'metadata':dict}`
- Document: https://docs.pinecone.io/reference/api/2024-07/data-plane/upsert

In [ ]:
# Create id
ids = [str(x) for x in range(len(documents))]
# Create metadata
metadatas = [{'text': text} for text in documents]
# Zip them together
records = zip(ids, doc_embeddings, metadatas)
index.upsert(vectors=records)   # There is a limit on how much you can upsert at a time. See https://docs.pinecone.io/guides/data/upsert-data

In [ ]:
index.describe_index_stats()

### Retriving Documents




In [ ]:
query = 'Can you tell me about the data science class?'

# 1) Embedding your query
embed_query = model.encode(query).tolist()
retrieved_docs =  index.query(vector=embed_query, top_k=1, include_metadata=True)
print(retrieved_docs)

In [ ]:
text = [r['metadata']['text'] for r in retrieved_docs['matches']]
print(text)

### Generate with retrived documents

In [ ]:
def RAG_pinecone_response(query, top_k=1):
  # First embedding your query
  embed_query = model.encode(query).tolist()
  # Then retrieve the document
  retrieved_docs =  index.query(vector=embed_query,
                                top_k=top_k,
                                include_metadata=True)
  # Then get the actual text
  text = [r['metadata']['text'] for r in retrieved_docs['matches']]
  # Finally join them together
  context = "\n".join(text)

  prompt = f'''<question>{query}</question>
  Please use context in <context> tags to answr the question.
  <context>{context}</context>'''
  response = get_response(prompt)
  # printing out
  print('Query:', query)
  print('Retrieved documents:', text)
  print('Response:', response)

query = 'Can you tell me about the data science class?'
RAG_pinecone_response(query)